In [2]:
# using ForwardDiff
using DataFrames
using LinearAlgebra

using Pkg

using Ipopt
using JuMP

In [3]:
using Interpolations

In [5]:
# updated for solving MPC with TES task to use TES all the time

function chargetime(time) # charge = 1, discharge = -1
    xp = [0, 7.999, 8, 25]
    fp = [1,1,0,0]
    f =  LinearInterpolation(xp, fp,extrapolation_bc=Flat())
    return f(time)
    end

function dischargetime(time) # charge = 1, discharge = -1
    xp = [0, 8.999, 9, 17.999, 18, 25]
    fp = [0,0,-1,-1,0,0]
    f = LinearInterpolation(xp, fp,extrapolation_bc=Flat())
    return f(time)
    end

dischargetime (generic function with 1 method)

In [6]:
chargetime(8)

0.0

In [7]:
# PRICES FOR ELECTRICITY
function price(time, type_price="commercial_1")
    if type_price == "individual_2"
        # индивидуальный тариф 2ставочный
        xp = [0, 7.999, 8 , 23.999, 24, 25]
        fp = [2.29, 2.29, 6.18, 6.18, 2.29, 2.29]
        f = LinearInterpolation(xp, fp,extrapolation_bc=Flat())
        return f(time)
    end
    if type_price == "individual_3"
        # индивидуальный тариф 3ставочный
        xp = [0, 7.999, 8 , 10.999, 11, 17.999, 18, 21.999, 22, 23.999, 24, 25]
        fp = [2.29, 2.29, 7, 7, 5.38, 5.38, 7, 7, 5.38, 5.38, 2.29, 2.29]
        f = LinearInterpolation(xp, fp,extrapolation_bc=Flat())
        return f(time)
    end
    if type_price == "commercial_1"
        # коммерческий тариф 4к
        xp = [0, 0.999, 1, 1.999, 2, 2.999, 3, 3.999, 4, 4.999, 5, 5.999, 6, 6.999, 7, 7.999, 8, 8.999, 9, 9.999, 10, 10.999, 11, 11.999, 12, 12.999, 13, 13.999, 14, 14.999, 15, 15.999, 16, 16.999, 17, 17.999, 18, 18.999, 19, 19.999, 20, 20.999, 21, 21.999, 22, 22.999, 23, 23.999, 24, 24.999]
        fp = [4.41704, 4.41704, 4.25404, 4.25404, 4.14436, 4.14436, 4.05468, 4.05468, 4.01459, 4.01459, 4.02631, 4.02631, 4.16135, 4.16135, 4.24251, 4.24251, 4.49976, 4.49976, 4.59363, 4.59363, 4.77394, 4.77394, 4.80014, 4.80014, 4.78849, 4.78849, 4.75757, 4.75757, 4.77814, 4.77814, 4.79783, 4.79783, 4.7674, 4.7674, 4.73724, 4.73724, 4.72584, 4.72584, 4.73167, 4.73167, 4.73894, 4.73894, 4.7544, 4.7544, 4.75318, 4.75318, 4.61478, 4.61478, 4.41704, 4.41704]
        f = LinearInterpolation(xp, fp,extrapolation_bc=Flat())
        return f(time)
    end
    if type_price == "commercial_2"
        # коммерческий тариф 2к
        xp = [0, 3, 6 , 9, 12, 15, 18, 21, 25]
        fp = [0, 0, 1, 2, 3, 4, 4, 3, 3]
        f = LinearInterpolation(xp, fp,extrapolation_bc=Flat())
        return f(time)
    end
    end

price (generic function with 2 methods)

In [10]:
price(14)
LinearInterpolation([0, 0.999, 1, 1.999, 2, 2.999, 3, 3.999, 4, 4.999, 5, 5.999, 6, 6.999, 7, 7.999, 8, 8.999, 9, 9.999, 10, 10.999, 11, 11.999, 12, 12.999, 13, 13.999, 14, 14.999, 15, 15.999, 16, 16.999, 17, 17.999, 18, 18.999, 19, 19.999, 20, 20.999, 21, 21.999, 22, 22.999, 23, 23.999, 24, 24.999], [4.41704, 4.41704, 4.25404, 4.25404, 4.14436, 4.14436, 4.05468, 4.05468, 4.01459, 4.01459, 4.02631, 4.02631, 4.16135, 4.16135, 4.24251, 4.24251, 4.49976, 4.49976, 4.59363, 4.59363, 4.77394, 4.77394, 4.80014, 4.80014, 4.78849, 4.78849, 4.75757, 4.75757, 4.77814, 4.77814, 4.79783, 4.79783, 4.7674, 4.7674, 4.73724, 4.73724, 4.72584, 4.72584, 4.73167, 4.73167, 4.73894, 4.73894, 4.7544, 4.7544, 4.75318, 4.75318, 4.61478, 4.61478, 4.41704, 4.41704],extrapolation_bc=Flat())

50-element extrapolate(interpolate((::Array{Float64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 4.41704
 4.41704
 4.25404
 4.25404
 4.14436
 4.14436
 4.05468
 4.05468
 4.01459
 4.01459
 4.02631
 4.02631
 4.16135
 ⋮      
 4.73167
 4.73167
 4.73894
 4.73894
 4.7544 
 4.7544 
 4.75318
 4.75318
 4.61478
 4.61478
 4.41704
 4.41704

In [11]:
# OCCUPANCY PROFILE
function occup(time)
    xp = [0, 8.9, 9, 12, 12.1, 15, 15.1, 18, 18.1]
    fp = [0, 0,   20,20, 30,   30, 10,    10,  0]
    n_hum = LinearInterpolation(xp, fp,extrapolation_bc=Flat())
    return n_hum(time)
end

occup (generic function with 1 method)

In [12]:
# occup(11)
LinearInterpolation([0, 8.9, 9, 12, 12.1, 15, 15.1, 18, 18.1], [0, 0,   20,20, 30,   30, 10,    10,  0],extrapolation_bc=Flat())(11)

20.0

In [13]:
using Dierckx
using Gadfly

In [16]:
# TEMPERATURE EXAMPLES
function outtemp_example(time, day="B")
    # https://msk.nuipogoda.ru/25-июля-2018
    if day == "A" 
        # зимний день 26.02.2018 Москва
        xp = [0, 3, 6 , 9, 12, 15, 18, 21, 25]
        fp = [-17, -19, -19, -18, -16, -14, -15, -19, -20]
        f = Spline1D(xp, fp)
        return f(time)
    end
    if day == "B"
        # летний день 25.07.2018 Москва
        xp = [0, 3, 6 , 9, 12, 15, 18, 21, 25]
        fp = [19, 17, 18, 20, 25, 23, 21, 19, 18]
        f = Spline1D(xp, fp)
        return f(time)
    end
    if day == "C"
        # осенний день 10.10.2015 Москва
        xp = [0, 3, 6 , 9, 12, 15, 18, 21, 25]
        fp = [0, 0, 1, 2, 3, 4, 4, 3, 3]
        f = Spline1D(xp, fp)
        return f(time)
    end
    if day == "D"
        # весенний день 20.03.2014 Москва
        xp = [0, 3, 6 , 9, 12, 15, 18, 21, 25]
        fp = [-5, -7, -9, -5, -4, -4, -3, -5, -8]
        f = Spline1D(xp, fp)
        return f(time)
    end
end

outtemp_example (generic function with 2 methods)

In [17]:
outtemp_example(1)

17.734737903985

In [18]:
Spline1D([0, 3, 6 , 9, 12, 15, 18, 21, 25], [-17, -19, -19, -18, -16, -14, -15, -19, -20])(1)

-17.96866851511626

In [19]:
tau = 60 #seconds
# N = 24*3600 / tau
# N = round(N)
N = 1440
time_t = zeros(1,N)
# time_t[i] = (i-1)*60/3600 for i=1:N
# prices_t = price(time_t)
COP = 3
C_out = 1005 # J/(kg K)
C_in = 1005 # J/(kg K)
m_air = 340 # kg
m_wall = 11520/2 #kg
C_wall = 1500 # J/(kg K)
pa1 = tau/(m_air*C_in)
h = 5 # m
b = 10 # m
k = 0.183 
pa2 = h*b*k
alpha_int = 8.7 # W/ m K
S = 60 # m^2
pa3 = 0.5 * alpha_int * h * (S / b + b)
M_met = 120 # W
T_vent = 21 # ^oC
a_inf = 0.02 / 3600
pa4 = a_inf * m_air * C_out
M_light = 20 # W
N_light = 15
W_light = M_light * N_light
L_evap = 333000 # W/kg
M_moist = 0.000021 # kg/s
pa5 = M_met - L_evap * M_moist
pw1 = tau/(m_wall*C_wall)
pn1 = tau/m_air
pn2 = a_inf * m_air
k_CO2 = 0.038 * 4 # g / s # medium work
ppm_k_CO2 = k_CO2 / 1000 / (S * h) * 1000000 # mass to kg divide by volume and to ppm
T_output = 18 # outer air temperature after TES

18

In [20]:
using Juniper

In [21]:
optimizer = Juniper.Optimizer
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0)

MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[RawParameter("print_level")=>0])

In [22]:
m1 = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver))

v = [10,20,12,23,42]
w = [12,45,12,22,21]
@variable(m1, x[1:5], Bin)

@objective(m1, Max, dot(v,x))

@NLconstraint(m1, sum(w[i]*x[i]^2 for i=1:5) <= 45)   

optimize!(m1)

# retrieve the objective value, corresponding x values and the status
println(JuMP.value.(x))
println(JuMP.objective_value(m1))
println(JuMP.termination_status(m1))

nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[RawParameter("print_level")=>0])
feasibility_pump  : false
log_levels        : Symbol[:Options, :Table, :Info]

#Variables: 5
#IntBinVar: 5
#Constraints: 1
#Linear Constraints: 0
#Quadratic Constraints: 0
#NonLinear Constraints: 1
Obj Sense: Max


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Incumbent using start values: 0.0
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 34.65599989891052
Relaxation Obj: 77.75505908623647

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time  

In [ ]:
# Quick start is ok. Juniper is working

In [23]:
mm = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Juniper

In [24]:
@variable(mm, P_c, base_name="compressor", lower_bound=0, upper_bound=3, Int)
@variable(mm, P_f, base_name="fan", lower_bound=0, upper_bound=10, Int)

fan

In [25]:
@objective(mm, Min, 10*P_f + 500 * P_c)
# @objective(mm, Min, sum(w[i]*x[i]^2 for i=1:N))

10 fan + 500 compressor

In [26]:
@constraint(mm, con1, 10*P_f + 500*P_c >= 600)

con1 : 10 fan + 500 compressor >= 600.0

In [27]:
@NLconstraint(mm,nl1, P_f*P_c >= 1)

fan * compressor - 1.0 >= 0

In [28]:
optimize!(mm)

nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[RawParameter("print_level")=>0])
feasibility_pump  : false
log_levels        : Symbol[:Options, :Table, :Info]

#Variables: 2
#IntBinVar: 2
#Constraints: 2
#Linear Constraints: 1
#Quadratic Constraints: 0
#NonLinear Constraints: 1
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 1.9730000495910645
Relaxation Obj: 599.9999940125292

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    0       2           599.999994009               599.999994013         0.0%    0.1       0         -     

#branches: 1
Obj: 599.9999940086939


In [29]:
println(JuMP.value.(P_c))
println(JuMP.value.(P_f))
println(JuMP.objective_value(mm))
println(JuMP.termination_status(mm))

0.9999999974378544
9.999999528976666
599.9999940086939
LOCALLY_SOLVED


In [ ]:
# Juniper solved this (above)

In [30]:
m2 = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Juniper

In [31]:
@variable(m2, P_c[1:1440], base_name="compressor", lower_bound=0, upper_bound=3, Int)
@variable(m2, P_f[1:1440], base_name="fan", lower_bound=0, upper_bound=10, Int)

1440-element Array{VariableRef,1}:
 fan[1]   
 fan[2]   
 fan[3]   
 fan[4]   
 fan[5]   
 fan[6]   
 fan[7]   
 fan[8]   
 fan[9]   
 fan[10]  
 fan[11]  
 fan[12]  
 fan[13]  
 ⋮        
 fan[1429]
 fan[1430]
 fan[1431]
 fan[1432]
 fan[1433]
 fan[1434]
 fan[1435]
 fan[1436]
 fan[1437]
 fan[1438]
 fan[1439]
 fan[1440]

In [32]:
@variable(m2, W_hvac[1:1440], lower_bound=0, upper_bound=10000)
# @variable(m2, W_dis[1:1440], lower_bound=0, upper_bound=10000)
# @variable(m2, W_ch[1:1440], lower_bound=0, upper_bound=10000)

1440-element Array{VariableRef,1}:
 W_hvac[1]   
 W_hvac[2]   
 W_hvac[3]   
 W_hvac[4]   
 W_hvac[5]   
 W_hvac[6]   
 W_hvac[7]   
 W_hvac[8]   
 W_hvac[9]   
 W_hvac[10]  
 W_hvac[11]  
 W_hvac[12]  
 W_hvac[13]  
 ⋮           
 W_hvac[1429]
 W_hvac[1430]
 W_hvac[1431]
 W_hvac[1432]
 W_hvac[1433]
 W_hvac[1434]
 W_hvac[1435]
 W_hvac[1436]
 W_hvac[1437]
 W_hvac[1438]
 W_hvac[1439]
 W_hvac[1440]

In [33]:
@variable(m2, Q[1:1440], lower_bound=0, upper_bound=100)
# @variable(m2, E[1:1440], lower_bound=0, upper_bound=10000000)
@variable(m2, T_r[1:1440], lower_bound=0, upper_bound=50)
@variable(m2, T_w[1:1440], lower_bound=0, upper_bound=50)
@variable(m2, n_co2[1:1440], lower_bound=0, upper_bound=2000)

1440-element Array{VariableRef,1}:
 n_co2[1]   
 n_co2[2]   
 n_co2[3]   
 n_co2[4]   
 n_co2[5]   
 n_co2[6]   
 n_co2[7]   
 n_co2[8]   
 n_co2[9]   
 n_co2[10]  
 n_co2[11]  
 n_co2[12]  
 n_co2[13]  
 ⋮          
 n_co2[1429]
 n_co2[1430]
 n_co2[1431]
 n_co2[1432]
 n_co2[1433]
 n_co2[1434]
 n_co2[1435]
 n_co2[1436]
 n_co2[1437]
 n_co2[1438]
 n_co2[1439]
 n_co2[1440]

In [34]:
@expression(m2, expr_price[i = 1:1440],LinearInterpolation([0, 0.999, 1, 1.999, 2, 2.999, 3, 3.999, 4, 4.999, 5, 5.999, 6, 6.999, 7, 7.999, 8, 8.999, 9, 9.999, 10, 10.999, 11, 11.999, 12, 12.999, 13, 13.999, 14, 14.999, 15, 15.999, 16, 16.999, 17, 17.999, 18, 18.999, 19, 19.999, 20, 20.999, 21, 21.999, 22, 22.999, 23, 23.999, 24, 24.999], [4.41704, 4.41704, 4.25404, 4.25404, 4.14436, 4.14436, 4.05468, 4.05468, 4.01459, 4.01459, 4.02631, 4.02631, 4.16135, 4.16135, 4.24251, 4.24251, 4.49976, 4.49976, 4.59363, 4.59363, 4.77394, 4.77394, 4.80014, 4.80014, 4.78849, 4.78849, 4.75757, 4.75757, 4.77814, 4.77814, 4.79783, 4.79783, 4.7674, 4.7674, 4.73724, 4.73724, 4.72584, 4.72584, 4.73167, 4.73167, 4.73894, 4.73894, 4.7544, 4.7544, 4.75318, 4.75318, 4.61478, 4.61478, 4.41704, 4.41704],extrapolation_bc=Flat())(i/60))

1440-element Array{Float64,1}:
 4.41704          
 4.41704          
 4.417040000000001
 4.417039999999999
 4.41704          
 4.41704          
 4.41704          
 4.41704          
 4.41704          
 4.41704          
 4.41704          
 4.41704          
 4.41704          
 ⋮                
 4.61478          
 4.61478          
 4.61478          
 4.61478          
 4.61478          
 4.61478          
 4.61478          
 4.61478          
 4.61478          
 4.61478          
 4.61478          
 4.41704          

In [35]:
@expression(m2, expr_obj_init[i in 1:1440], 10 * P_f[i] + 500 * P_c[i])

1440-element Array{GenericAffExpr{Float64,VariableRef},1}:
 10 fan[1] + 500 compressor[1]      
 10 fan[2] + 500 compressor[2]      
 10 fan[3] + 500 compressor[3]      
 10 fan[4] + 500 compressor[4]      
 10 fan[5] + 500 compressor[5]      
 10 fan[6] + 500 compressor[6]      
 10 fan[7] + 500 compressor[7]      
 10 fan[8] + 500 compressor[8]      
 10 fan[9] + 500 compressor[9]      
 10 fan[10] + 500 compressor[10]    
 10 fan[11] + 500 compressor[11]    
 10 fan[12] + 500 compressor[12]    
 10 fan[13] + 500 compressor[13]    
 ⋮                                  
 10 fan[1429] + 500 compressor[1429]
 10 fan[1430] + 500 compressor[1430]
 10 fan[1431] + 500 compressor[1431]
 10 fan[1432] + 500 compressor[1432]
 10 fan[1433] + 500 compressor[1433]
 10 fan[1434] + 500 compressor[1434]
 10 fan[1435] + 500 compressor[1435]
 10 fan[1436] + 500 compressor[1436]
 10 fan[1437] + 500 compressor[1437]
 10 fan[1438] + 500 compressor[1438]
 10 fan[1439] + 500 compressor[1439]
 10 fan[1440] + 

In [37]:
@expression(m2, expr_obj, sum(expr_obj_init[j] * expr_price[j] for j in 1:1440))

44.1704 fan[1] + 2208.52 compressor[1] + 44.1704 fan[2] + 2208.52 compressor[2] + 44.17040000000001 fan[3] + 2208.5200000000004 compressor[3] + 44.170399999999994 fan[4] + 2208.5199999999995 compressor[4] + 44.1704 fan[5] + 2208.52 compressor[5] + 44.1704 fan[6] + 2208.52 compressor[6] + 44.1704 fan[7] + 2208.52 compressor[7] + 44.1704 fan[8] + 2208.52 compressor[8] + 44.1704 fan[9] + 2208.52 compressor[9] + 44.1704 fan[10] + 2208.52 compressor[10] + 44.1704 fan[11] + 2208.52 compressor[11] + 44.1704 fan[12] + 2208.52 compressor[12] + 44.1704 fan[13] + 2208.52 compressor[13] + 44.1704 fan[14] + 2208.52 compressor[14] + 44.1704 fan[15] + 2208.52 compressor[15] + 44.1704 fan[16] + 2208.52 compressor[16] + 44.1704 fan[17] + 2208.52 compressor[17] + 44.1704 fan[18] + 2208.52 compressor[18] + 44.1704 fan[19] + 2208.52 compressor[19] + 44.1704 fan[20] + 2208.52 compressor[20] + 44.1704 fan[21] + 2208.52 compressor[21] + 44.1704 fan[22] + 2208.52 compressor[22] + 44.1704 fan[23] + 2208.52 com

In [38]:
@objective(m2, Min, expr_obj)

44.1704 fan[1] + 2208.52 compressor[1] + 44.1704 fan[2] + 2208.52 compressor[2] + 44.17040000000001 fan[3] + 2208.5200000000004 compressor[3] + 44.170399999999994 fan[4] + 2208.5199999999995 compressor[4] + 44.1704 fan[5] + 2208.52 compressor[5] + 44.1704 fan[6] + 2208.52 compressor[6] + 44.1704 fan[7] + 2208.52 compressor[7] + 44.1704 fan[8] + 2208.52 compressor[8] + 44.1704 fan[9] + 2208.52 compressor[9] + 44.1704 fan[10] + 2208.52 compressor[10] + 44.1704 fan[11] + 2208.52 compressor[11] + 44.1704 fan[12] + 2208.52 compressor[12] + 44.1704 fan[13] + 2208.52 compressor[13] + 44.1704 fan[14] + 2208.52 compressor[14] + 44.1704 fan[15] + 2208.52 compressor[15] + 44.1704 fan[16] + 2208.52 compressor[16] + 44.1704 fan[17] + 2208.52 compressor[17] + 44.1704 fan[18] + 2208.52 compressor[18] + 44.1704 fan[19] + 2208.52 compressor[19] + 44.1704 fan[20] + 2208.52 compressor[20] + 44.1704 fan[21] + 2208.52 compressor[21] + 44.1704 fan[22] + 2208.52 compressor[22] + 44.1704 fan[23] + 2208.52 com

In [42]:
# @constraint(m2,con2, W_hvac - COP * 500 * P_c + W_dis - W_ch == 0)
@constraint(m2,con2[i in 1:1440], W_hvac[i] - COP * 500 * P_c[i] == 0)

1440-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},1}:
 con2[1] : W_hvac[1] - 1500 compressor[1] == 0.0         
 con2[2] : W_hvac[2] - 1500 compressor[2] == 0.0         
 con2[3] : W_hvac[3] - 1500 compressor[3] == 0.0         
 con2[4] : W_hvac[4] - 1500 compressor[4] == 0.0         
 con2[5] : W_hvac[5] - 1500 compressor[5] == 0.0         
 con2[6] : W_hvac[6] - 1500 compressor[6] == 0.0         
 con2[7] : W_hvac[7] - 1500 compressor[7] == 0.0         
 con2[8] : W_hvac[8] - 1500 compressor[8] == 0.0         
 con2[9] : W_hvac[9] - 1500 compressor[9] == 0.0         
 con2[10] : W_hvac[10] - 1500 compressor[10] == 0.0      
 con2[11] : W_hvac[11] - 1500 compressor[11] == 0.0      
 con2[12] : W_hvac[12] - 1500 compressor[12] == 0.0      
 con2[13] : W_hvac[13] - 1500 compressor[13] == 0.0      
 ⋮                                                       
 con2[1429] : W_h

In [43]:
@constraint(m2,con3[i in 1:1440], Q[i] - 0.55 * P_f[i] * 10 / 100 == 0)

1440-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},1}:
 con3[1] : Q[1] - 0.055 fan[1] == 0.0         
 con3[2] : Q[2] - 0.055 fan[2] == 0.0         
 con3[3] : Q[3] - 0.055 fan[3] == 0.0         
 con3[4] : Q[4] - 0.055 fan[4] == 0.0         
 con3[5] : Q[5] - 0.055 fan[5] == 0.0         
 con3[6] : Q[6] - 0.055 fan[6] == 0.0         
 con3[7] : Q[7] - 0.055 fan[7] == 0.0         
 con3[8] : Q[8] - 0.055 fan[8] == 0.0         
 con3[9] : Q[9] - 0.055 fan[9] == 0.0         
 con3[10] : Q[10] - 0.055 fan[10] == 0.0      
 con3[11] : Q[11] - 0.055 fan[11] == 0.0      
 con3[12] : Q[12] - 0.055 fan[12] == 0.0      
 con3[13] : Q[13] - 0.055 fan[13] == 0.0      
 ⋮                                            
 con3[1429] : Q[1429] - 0.055 fan[1429] == 0.0
 con3[1430] : Q[1430] - 0.055 fan[1430] == 0.0
 con3[1431] : Q[1431] - 0.055 fan[1431] == 0.0
 con3[1432] : Q[1432] - 0.055 

In [44]:
# @constraint(m2,con4, E[1] == 0)
@constraint(m2,con5, T_r[1] == 19)
@constraint(m2,con6, T_w[1] == 19)
@constraint(m2,con7, n_co2[1] == 400)

con7 : n_co2[1] == 400.0

In [45]:
@constraint(m2,con8[i in 1:1440], T_r[i] >= 18)
@constraint(m2,con9[i in 1:1440], T_r[i] <= 24)
@constraint(m2,con10[i in 1:1440], n_co2[i] <= 800)

1440-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 con10[1] : n_co2[1] <= 800.0      
 con10[2] : n_co2[2] <= 800.0      
 con10[3] : n_co2[3] <= 800.0      
 con10[4] : n_co2[4] <= 800.0      
 con10[5] : n_co2[5] <= 800.0      
 con10[6] : n_co2[6] <= 800.0      
 con10[7] : n_co2[7] <= 800.0      
 con10[8] : n_co2[8] <= 800.0      
 con10[9] : n_co2[9] <= 800.0      
 con10[10] : n_co2[10] <= 800.0    
 con10[11] : n_co2[11] <= 800.0    
 con10[12] : n_co2[12] <= 800.0    
 con10[13] : n_co2[13] <= 800.0    
 ⋮                                 
 con10[1429] : n_co2[1429] <= 800.0
 con10[1430] : n_co2[1430] <= 800.0
 con10[1431] : n_co2[1431] <= 800.0
 con10[1432] : n_co2[1432] <= 800.0
 con10[1433] : n_co2[1433] <= 800.0
 con10[1434] : n_co2[1434] <= 800.0
 con10[1435] : n_co2[1435] <= 800.0
 con10[1436] : n_co2[1436] <= 800.0
 con10[1437] : n_co2[1437] <= 800.0


In [46]:
@expression(m2, expr_out[i = 1:1440], Spline1D([0, 3, 6 , 9, 12, 15, 18, 21, 25], [-17, -19, -19, -18, -16, -14, -15, -19, -20])(i/60))

1440-element Array{Float64,1}:
 -17.01900928328442 
 -17.037917169970726
 -17.056723881669004
 -17.07542963998933 
 -17.09403466654177 
 -17.112539182936395
 -17.13094341078329 
 -17.149247571692506
 -17.16745188727414 
 -17.185556579138247
 -17.203561868894916
 -17.221467978154205
 -17.239275128526195
   ⋮                
 -20.9635603093405  
 -20.95944908878232 
 -20.955092084463015
 -20.950488231016415
 -20.945636463076355
 -20.940535715276663
 -20.935184922251175
 -20.929583018633725
 -20.923728939058144
 -20.917621618158265
 -20.911259990567913
 -20.904642990920927

In [47]:
@expression(m2, expr_occup[i = 1:1440], LinearInterpolation([0, 8.9, 9, 12, 12.1, 15, 15.1, 18, 18.1], [0, 0,   20,20, 30,   30, 10,    10,  0],extrapolation_bc=Flat())(i/60))

1440-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [48]:
@NLconstraint(m2,nl1[i in 1:1439], T_r[i+1] - (T_r[i] + pa1*((pa2 + pa4) * (expr_out[i] - T_r[i]) + pa3 * (T_w[i] - T_r[i]) + pa5 * expr_occup[i] - W_hvac[i] + C_out * T_vent * Q[i] - C_in * T_r[i] * Q[i] + W_light)) == 0)

1439-element Array{ConstraintRef{Model,NonlinearConstraintIndex,ScalarShape},1}:
 (T_r[2] - (T_r[1] + 0.0001755926251097454 * ((((((9.15 + 1.8983333333333334) * (-17.01900928328442 - T_r[1]) + 348.0 * (T_w[1] - T_r[1]) + 113.007 * 0.0) - W_hvac[1]) + 1005.0 * 21.0 * Q[1]) - 1005.0 * T_r[1] * Q[1]) + 300.0))) - 0.0 == 0                            
 (T_r[3] - (T_r[2] + 0.0001755926251097454 * ((((((9.15 + 1.8983333333333334) * (-17.037917169970726 - T_r[2]) + 348.0 * (T_w[2] - T_r[2]) + 113.007 * 0.0) - W_hvac[2]) + 1005.0 * 21.0 * Q[2]) - 1005.0 * T_r[2] * Q[2]) + 300.0))) - 0.0 == 0                           
 (T_r[4] - (T_r[3] + 0.0001755926251097454 * ((((((9.15 + 1.8983333333333334) * (-17.056723881669004 - T_r[3]) + 348.0 * (T_w[3] - T_r[3]) + 113.007 * 0.0) - W_hvac[3]) + 1005.0 * 21.0 * Q[3]) - 1005.0 * T_r[3] * Q[3]) + 300.0))) - 0.0 == 0                           
 (T_r[5] - (T_r[4] + 0.0001755926251097454 * ((((((9.15 + 1.8983333333333334) * (-17.07542963998933 - T_r[4]) + 348

In [49]:
@constraint(m2,con11[i in 1:1439], T_w[i+1] - (T_w[i] + pw1 * pa2 * (T_r[i] - T_w[i])) == 0)

1439-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},1}:
 con11[1] : T_w[2] - 0.9999364583333333 T_w[1] - 6.354166666666668e-5 T_r[1] == 0.0            
 con11[2] : T_w[3] - 0.9999364583333333 T_w[2] - 6.354166666666668e-5 T_r[2] == 0.0            
 con11[3] : T_w[4] - 0.9999364583333333 T_w[3] - 6.354166666666668e-5 T_r[3] == 0.0            
 con11[4] : T_w[5] - 0.9999364583333333 T_w[4] - 6.354166666666668e-5 T_r[4] == 0.0            
 con11[5] : T_w[6] - 0.9999364583333333 T_w[5] - 6.354166666666668e-5 T_r[5] == 0.0            
 con11[6] : T_w[7] - 0.9999364583333333 T_w[6] - 6.354166666666668e-5 T_r[6] == 0.0            
 con11[7] : T_w[8] - 0.9999364583333333 T_w[7] - 6.354166666666668e-5 T_r[7] == 0.0            
 con11[8] : T_w[9] - 0.9999364583333333 T_w[8] - 6.354166666666668e-5 T_r[8] == 0.0            
 con11[9] : T_w[10] - 0.9999364583333333 T_w[9] - 6.354166666

In [50]:
@constraint(m2,con12[i in 1:1439], n_co2[i+1] - (n_co2[i] + pn1 * (Q[i] + pn2) * (300 - n_co2[i]) + k_CO2 * expr_occup[i]) == 0)

1439-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},1}:
 con12[1] : 0.17647058823529413 Q[1]*n_co2[1] + n_co2[2] - 0.9996666666666667 n_co2[1] - 52.94117647058824 Q[1] == 0.10000000000000002                  
 con12[2] : 0.17647058823529413 Q[2]*n_co2[2] + n_co2[3] - 0.9996666666666667 n_co2[2] - 52.94117647058824 Q[2] == 0.10000000000000002                  
 con12[3] : 0.17647058823529413 Q[3]*n_co2[3] + n_co2[4] - 0.9996666666666667 n_co2[3] - 52.94117647058824 Q[3] == 0.10000000000000002                  
 con12[4] : 0.17647058823529413 Q[4]*n_co2[4] + n_co2[5] - 0.9996666666666667 n_co2[4] - 52.94117647058824 Q[4] == 0.10000000000000002                  
 con12[5] : 0.17647058823529413 Q[5]*n_co2[5] + n_co2[6] - 0.9996666666666667 n_co2[5] - 52.94117647058824 Q[5] == 0.10000000000000002                  
 con12[6] : 0.17647058823529413 Q[6]*n_co2[6] + n_co2[7] - 0.

In [51]:
optimize!(m2)

nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[RawParameter("print_level")=>0])
feasibility_pump  : false
log_levels        : Symbol[:Options, :Table, :Info]

#Variables: 10080
#IntBinVar: 2880
#Constraints: 11520
#Linear Constraints: 8642
#Quadratic Constraints: 1439
#NonLinear Constraints: 1439
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 361.1560001373291
Relaxation Obj: 96636.44812700518

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    2       2                 -                        96636.45             -    103.9      0         -     

┌ Info: Breaking out of strong branching as the time limit of 100.0 seconds got reached.
└ @ Juniper C:\Users\Оля\.julia\packages\Juniper\atNbZ\src\bb_strategies.jl:237



    3       3                 -                        96636.68             -    109.0      -       74.9%   
    4       3                 -                        96639.75             -    111.8      -       76.6%   
    5       4                 -                        96640.12             -    115.9      -       82.0%   
    6       4                 -                        96640.3              -    119.7      -       77.3%   
    7       4                 -                        96643.45             -    123.7      -       77.3%   
    8       4                 -                        96643.68             -    126.6      -       82.6%   
    9       5                 -                        96643.82             -    130.8      -       76.3%   
   10       5                 -                        96643.99             -    134.6      -       77.3%   
   11       5                 -                        96644.47             -    139.0      -       75.9%   
   12       5     

   153      9                 -                        96659.95             -    735.5      -       78.9%   
   154      9                 -                        96659.96             -    739.8      -       79.6%   
   155      9                 -                        96660.03             -    744.0      -       78.9%   
   156      9                 -                        96660.04             -    748.2      -       60.9%   
   157      9                 -                        96660.07             -    752.2      -       93.8%   


┌ Warning: Only almost solved
└ @ Juniper C:\Users\Оля\.julia\packages\Juniper\atNbZ\src\BnBTree.jl:89


...

  4605      15                -                        96675.62             -   23163.2     -       97.7%   
  4606      13                -                        96675.62             -   23168.5     -       95.6%   
  4607      15                -                        96675.62             -   23173.4     -       97.8%   
  4608      15                -                        96675.62             -   23179.8     -       97.6%   
  4609      15                -                        96675.62             -   23185.5     -       97.4%   
  4610      14                -                        96675.62             -   23190.9     -       95.2%   
  4611      14                -                        96675.62             -   23195.2     -       94.7%   
  4612      15                -                        96675.62             -   23201.0     -       97.7%   
  4613      14                -                        96675.62             -   23205.3     -       95.2%   
  4614      16     

┌ Warning: Only almost solved
└ @ Juniper C:\Users\Оля\.julia\packages\Juniper\atNbZ\src\BnBTree.jl:89


  4638      15                -                        96675.64             -   23354.5     -       96.6%   
  4639      14                -                        96675.64             -   23360.3     -       97.8%   
  4640      14                -                        96675.64             -   23366.3     -       95.7%   

Excessive output truncated after 524383 bytes.

InterruptException: InterruptException:

In [ ]:
# Juniper di not solve this for 24 hours